In [3]:
import pandas as pd
from siuba import *
import altair as alt

from shared_utils import geography_utils, calitp_color_palette as cp, styleguide

from dla_utils import _dla_utils


In [4]:
alt.themes.register("calitp_theme", styleguide.calitp_theme)
# enable
alt.themes.enable("calitp_theme")

ThemeRegistry.enable('calitp_theme')

In [5]:
%%html
<style>
@import url('https://fonts.googleapis.com/css?family=Lato');
</style>


In [6]:
#df = _clean_data.make_clean_data()
df= pd.read_parquet("gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/dla_df.parquet")

In [7]:
df.sample(1)

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
7262,Obligated,STPLN,5055(183),Anaheim,2016-01-21,2016-01-21,2016-02-02,2016-02-29,2016-03-07,-164653,...,0.00,Large,0,0,0,1,0,1,0,2


## Function to Count by Year

In [8]:
df_years = _dla_utils.count_all_years((df>>filter(_.dist==7)))

In [9]:
#from Tiffany's branch DLA functions
def count_all_years(df, groupedby=["dist"]):
    count_years = geography_utils.aggregate_by_geography(
        df, 
        group_cols = ["prepared_y", "dist"],
        sum_cols = ["total_requested", "ac_requested", "fed_requested"],
        mean_cols = ["total_requested", "ac_requested", "fed_requested"],
        nunique_cols = ["primary_agency_name", "prefix", "project_no", "project_location", "type_of_work"]
    ).sort_values(["prepared_y", "dist"], ascending=[False, True])
    
    count_years= count_years.rename(columns={"ac_requested_x": "ac_requested_sum",
                                "fed_requested_x": "fed_requested_sum",
                                "total_requested_x": "total_requested_sum",
                                "ac_requested_y": "ac_requested_mean",
                                "fed_requested_y": "fed_requested_mean",
                                "total_requested_y": "total_requested_mean",
                                "status": "counts",
                                "prefix": "unique_prefix",
                                "primary_agency_name": "unique_primary_agency_name",
                                "project_location": "unique_project_location",
                                "project_no":"unique_project_no",
                                "type_of_work":"unique_type_of_work"})

    return count_years

In [10]:
df1 = count_all_years(df)

In [11]:
df1

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
135,"2,023.00",1,"-6,498,042.00","56,107,618.00","67,964,868.00","-33,495.06","289,214.53","350,334.37",17.00,10.00,137.00,136.00,76.00
136,"2,023.00",2,"4,498,606.00","38,159,219.00","51,082,987.00","70,290.72","596,237.80","798,171.67",18.00,9.00,51.00,51.00,43.00
137,"2,023.00",3,"-21,356,438.00","217,723,765.00","261,445,051.00","-105,204.13","1,072,530.86","1,287,906.66",33.00,36.00,160.00,157.00,136.00
138,"2,023.00",4,"68,963,358.00","287,372,905.00","513,234,208.00","236,987.48","987,535.76","1,763,691.44",39.00,80.00,232.00,231.00,201.00
139,"2,023.00",5,"86,249,123.00","71,571,415.00","156,997,052.00","352,037.24","292,128.22","640,804.29",17.00,18.00,212.00,211.00,97.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df2 = count_all_years((df>>filter(_.dist==4)))

In [13]:
df2

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
12,"2,023.00",4,68963358,287372905,513234208,"236,987.48","987,535.76","1,763,691.44",39,80,232,231,201
11,"2,022.00",4,-32264811,342782897,245130910,"-92,982.16","987,846.97","706,429.14",36,96,264,279,252
10,"2,021.00",4,-7581871,151107792,209378112,"-18,674.56","372,186.68","515,709.64",42,101,316,326,288
9,"2,020.00",4,-16015854,219797497,385650237,"-55,418.18","760,544.97","1,334,429.89",34,89,234,243,215
8,"2,019.00",4,-70929401,314851257,319675522,"-177,323.50","787,128.14","799,188.81",41,93,322,333,299
0,"2,018.00",4,-19612826,286154006,374039028,"-43,200.06","630,295.17","823,874.51",51,97,344,369,315
1,"2,017.00",4,-12268569,279804750,332810636,"-33,429.34","762,410.76","906,840.97",43,97,292,319,269
2,"2,016.00",4,59687248,304782100,502320628,"144,172.10","736,188.65","1,213,334.85",50,113,336,361,300
3,"2,015.00",4,68364142,239952075,448655374,"152,258.67","534,414.42","999,232.46",48,109,374,394,300
4,"2,014.00",4,-21667366,178332665,229278381,"-44,491.51","366,186.17","470,797.50",52,108,376,400,295


In [14]:
df3= count_all_years((df>>filter(_.prefix=="CML")))

In [15]:
df3

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
99,"2,023.00",3,26434044,35850525,101802668,"944,073.00","1,280,375.89","3,635,809.57",1,12,23,23,24
104,"2,023.00",4,4960000,15988689,33425391,"177,142.86","571,024.61","1,193,763.96",1,18,26,25,26
108,"2,023.00",5,0,-646949,-730768,0.00,"-323,474.50","-365,384.00",1,1,2,2,1
100,"2,023.00",6,546000,39042032,57250148,"6,134.83","438,674.52","643,260.09",1,23,70,66,63
103,"2,023.00",7,0,8577445,16307992,0.00,"357,393.54","679,499.67",1,6,19,18,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,"2,013.00",10,0,-19261,-191630,0.00,"-9,630.50","-95,815.00",1,2,2,2,2
55,"2,013.00",11,0,821000,927648,0.00,"273,666.67","309,216.00",1,3,3,3,3
58,"2,013.00",12,0,546814,620993,0.00,"182,271.33","206,997.67",1,1,3,3,1
49,"2,012.00",4,0,3842691,10576983,0.00,"1,280,897.00","3,525,661.00",1,2,3,3,3


In [16]:
count_all_years(df>>filter(_.primary_agency_name=="Humboldt County"))

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
10,"2,023.00",1,-7009921,44531823,42772081,"-51,167.31","325,049.80","312,204.97",11,1,90,90,48
9,"2,022.00",1,-57051,34257605,39769563,-460.09,"276,271.01","320,722.28",8,1,94,95,29
8,"2,021.00",1,4776051,4780542,10580567,"68,229.30","68,293.46","151,150.96",9,1,63,62,31
7,"2,020.00",1,-3136118,12204760,10262739,"-37,334.74","145,294.76","122,175.46",9,1,59,60,32
6,"2,019.00",1,4946135,14222121,21295244,"33,647.18","96,749.12","144,865.61",9,1,77,78,43
0,"2,018.00",1,177241,3015693,3600429,"1,036.50","17,635.63","21,055.14",8,1,110,111,37
1,"2,017.00",1,0,5158087,5531544,0.00,"184,217.39","197,555.14",8,1,24,24,16
3,"2,016.00",1,0,4649799,5028073,0.00,"140,903.00","152,365.85",10,1,29,30,21
2,"2,015.00",1,0,4659372,5363874,0.00,"89,603.31","103,151.42",8,1,35,35,22
4,"2,014.00",1,0,4554255,4981764,0.00,"89,299.12","97,681.65",9,1,45,45,23


## Chart Part 1: Aggregate the Data

In [17]:
def part1_chart(df, col, aggregate_by=["dist"]):
    aggfunc = (
        geography_utils.aggregate_by_geography(
            df[df[aggregate_by].notna()], 
            group_cols = [aggregate_by],
            nunique_cols = [col]))       
    return aggfunc

In [18]:
part1_chart(df, "type_of_work", aggregate_by="primary_agency_name")

,primary_agency_name,type_of_work
0,Humboldt County,105
1,Mendocino County,65
2,Sacramento County,86
3,Sutter County,12
4,Contra Costa County,62
...,...,...
614,Palos Verdes Est,1
615,San Diego Metropolitan Transit System,1
616,Corte Madera,1
617,Hillsborough,1


## Find Top Nuniques

In [19]:
## want to create a function that gets a new dataframe that will hold all the counts and top twenty 
def find_top(df):
    
    cols= [["prefix",
           "prepared_y",
           "status_comment",
           "project_location",
           "type_of_work",
           "seq",
           "mpo",
           "primary_agency_name"]]
    
    tops_1 = (df
          >>count(_.prefix)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_prefix"})
    tops_2 = (df
          >>count(_.prepared_y)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_prepared_y"})
    tops_3 = (df
          >>count(_.status_comment)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_status_comment"})
    tops_4 = (df
          >>count(_.project_location)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_project_location"})
    tops_5 = (df
          >>count(_.type_of_work)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_type_of_work"})
    tops_6 = (df
          >>count(_.mpo)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_mpo"})
    tops_7 = (df
          >>count(_.primary_agency_name)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_primary_agency_name"})
    
    
    test = tops_1.join(tops_2, how='outer')
    test2 = test.join(tops_3, how='outer')
    test3 = test2.join(tops_4, how='outer')
    test4 = test3.join(tops_5, how='outer')
    test5 = test4.join(tops_6, how='outer')
    tops = test5.join(tops_7, how='outer')
                       
    return tops

In [20]:
tops = find_top(df)
tops

,prefix,unique_prefix,prepared_y,unique_prepared_y,status_comment,unique_status_comment,project_location,unique_project_location,type_of_work,unique_type_of_work,mpo,unique_mpo,primary_agency_name,unique_primary_agency_name
0,HSIPL,3974,"2,018.00","2,820.00",Authorized,24030,NaN,520,Bridge Replacement (tc),856,SCAG,6109,Humboldt County,898
1,CML,3946,"2,014.00","2,724.00",..,174,San Francisco Bay Area,81,Bridge Replacement,652,MTC,3955,Santa Cruz County,642
2,STPL,3344,"2,019.00","2,655.00",Prog Code Z400,108,FTA transfer,61,FTA Transfer,520,NON-MPO,3288,Los Angeles,599
3,ER,2259,"2,015.00","2,575.00",Prog Code M400,76,Golden Gate Bridge,36,Permanent Restoration,417,SACOG,2216,Los Angeles County,540
4,BRLO,1750,"2,016.00","2,505.00",Prog Code Z230,48,Various Locations,36,Road Rehabilitation,300,CFCG,1589,Fresno,522
5,BRLS,1138,"2,020.00","2,336.00",Prog Code Z003,30,Sacog Region,32,Emergency Opening,278,KCOG,1169,El Dorado County,354
6,ATPL,748,"2,017.00","2,330.00",Prog Code M40E,27,Throughout Kern County,27,Bridge Preventive Maintenance,192,AMBAG,1022,Kern County,339
7,RPSTPL,618,"2,021.00","2,315.00",prog code Y230,24,Citywide,20,Storm Damage Restoration,177,STANCOG,943,Stockton,318
8,ACSTP,416,"2,022.00","2,275.00",Prog Code M0E3,23,Various Locations In San Diego County,17,Pavement Rehabilitation,144,TCAG,590,Fresno County,315
9,HPLUL,373,"2,023.00","2,040.00",Prog Code M23E,17,Sf Bay Area: Regionwide,16,Bridge Rehabilitation,133,SJCG,583,Bakersfield,308


In [31]:
year_cols = ['prepared_y']
for col in year_cols:
    tops[col] = tops[col].astype('Int64')

In [32]:
tops.prepared_y.dtypes

Int64Dtype()

In [38]:
_dla_utils.basic_bar_chart(tops, "prepared_y:O","unique_prepared_y:Q", "prepared_y", "subset")

alt.Chart(...)

In [39]:
df2.sample()

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
2,"2,016.00",4,59687248,304782100,502320628,"144,172.10","736,188.65","1,213,334.85",50,113,336,361,300


In [43]:
_dla_utils.basic_bar_chart(
    (df2), "prepared_y:O", "unique_prefix:Q", "prepared_y:O", "subset")

alt.Chart(...)

In [48]:
_dla_utils.basic_bar_chart(
    (tops),"unique_primary_agency_name:Q", "primary_agency_name:N", "primary_agency_name:N", "subset")


alt.Chart(...)

In [49]:
_dla_utils.basic_scatter_chart(df1, "unique_primary_agency_name", "total_requested_sum", "dist", "subset", "")

alt.Chart(...)

In [51]:
_dla_utils.basic_line_chart(df1, "prepared_y:O", "total_requested_sum", "subset")

alt.Chart(...)

In [52]:
_dla_utils.basic_line_chart(df1, "prepared_y", "ac_requested_sum", "subset")

alt.Chart(...)

#### Issues/what to look for:

* get a df of information not grouped by year with 
    - total requested
    - ac requested 
    - fed requested
* fix name for charts of find_top 


## Chart Function

### Old Function with filtering in the function

In [57]:

def basic_agg(df, col, aggregate_by=["dist"]):
    df1 = ((df >> group_by(_[aggregate_by]) >> summarize(n=_[col].nunique()) >> arrange(-_.n)).head(20))
    chart = (alt.Chart(df1)
             .mark_bar()
             .encode(
                 x=alt.X(aggregate_by, title=f"{_dla_utils.labeling(aggregate_by)}"),
                 y=alt.Y("n", title=f"Number of Unique {_dla_utils.labeling(col)}"),
                 color = alt.Color("n",
                                  scale=alt.Scale(
                                      range=cp.CALITP_SEQUENTIAL_COLORS),
                                      legend=alt.Legend(title=f"{(_dla_utils.labeling(col))}")
                                  ))
             .properties( 
                          title=f"Number of Unique {_dla_utils.labeling(col)} by {_dla_utils.labeling(aggregate_by)}")
    )

    return chart

In [58]:
basic_agg(df, "prefix", aggregate_by="primary_agency_name")

alt.Chart(...)

## Function for Selecting Calculating Method

In [59]:
def calculate_data(df, col, aggregate_by=["dist"], aggfunc="sum"):
    df1 = (df.groupby(aggregate_by)
          .agg({col: aggfunc})
          .reset_index()
          .sort_values(col, ascending=False)
         )
    #df = df[df[col] > 0]
    df1 = (df1>>arrange(-_[col])).head(20)
    return df1

In [60]:
calculate_data(df, "total_requested", aggregate_by="dist", aggfunc="sum")

,dist,total_requested
7,7,6245161101
4,4,3628519598
8,8,3385088490
12,12,3074321165
3,3,2052526156
6,6,1879679309
11,11,1481611569
10,10,1287506305
5,5,793364021
2,2,322118203


In [61]:
# #calcualte_sum2 might be more versatile
# def calculate_sum(df, col, agg_col):
#     dftest = geography_utils.aggregate_by_geography(
#         df, 
#         group_cols = [agg_col],
#         sum_cols = [col])
#     dftest = (dftest>>arrange(-_[col])).head(20)
#     #dftest = (dftest>>filter(_[col] >0))
#     return dftest


In [62]:
# calculate_sum(df, "total_requested", agg_col="dist")

## Adding to Charting Function

In [63]:
def test_chartfunc_sum(df, col, aggregate_by, aggfunc):
    
    df1 = (calculate_data(df, col, aggregate_by, aggfunc))
    
    chart = (alt.Chart(df1)
             .mark_bar()
             .encode(
                 x=alt.X(aggregate_by, title=_dla_utils.labeling(aggregate_by), sort=('-y')),
                 y=alt.Y(col, title=_dla_utils.labeling(col)),
                 color = alt.Color(col,
                                  scale=alt.Scale(
                                      range=cp.CALITP_SEQUENTIAL_COLORS),
                                      legend=alt.Legend(title=(_dla_utils.labeling(col)))
                                  ))
             .properties( 
                          title=f"Highest {_dla_utils.labeling(col)} {_dla_utils.labeling(aggfunc)}s by {_dla_utils.labeling(aggregate_by)}")
    )
    
    return chart



In [64]:
test_chartfunc_sum(df, "total_requested", aggregate_by="type_of_work", aggfunc="mean")

alt.Chart(...)

In [65]:
test_chartfunc_sum(df, "prefix", aggregate_by="primary_agency_name", aggfunc="nunique")

alt.Chart(...)

In [66]:
#making sure it is right
df>>group_by(_.primary_agency_name)>> summarize(n=_.prefix.nunique()) >> arrange(-_.n)

,primary_agency_name,n
276,Los Angeles,38
277,Los Angeles County,32
434,San Bernardino Associated Governments,28
71,Caltrans,25
534,Stockton,25
...,...,...
591,Weed,1
595,Western Shasta Resource Conservation District,1
602,Willits,1
603,Willows,1


## Grouping Charts

In [67]:
#groupby_col_x axis = prepared_y
#agg_by_col = dist or mpo 
#sum_col= primary_agency_name or prefix
 
def group_chart_nunique(df, groupby_col_x, agg_by_col, sum_col):
    dist_years1 = (df >> group_by(_[groupby_col_x], _[agg_by_col]) 
                   >> summarize(n=_[sum_col].nunique()) 
                   >> arrange(-_[groupby_col_x]))

    chart = alt.Chart(dist_years1).mark_bar().encode(
            column=(f"{agg_by_col}:N"),
            x=alt.X((f"{groupby_col_x}:O"), title=_dla_utils.labeling(groupby_col_x)),
            y=alt.Y('n:Q', title=(f"Number of Unique {_dla_utils.labeling(sum_col)}")),
            color = alt.Color((f"{agg_by_col}:N"), 
                                  scale=alt.Scale(
                                      range=cp.CALITP_SEQUENTIAL_COLORS),  
                                   legend=alt.Legend(title=(_dla_utils.labeling(sum_col)))
                                  )
                                  )
    #chart.save(f"./chart_outputs/grouped_{agg_by_col}_by_{groupby_col_x}.png")
    
    return chart


In [69]:
def group_chart_nunique_2(df, groupby_col_x, agg_by_col, sum_col):
    #without function
    # dist_years1 = (df >> group_by(_[groupby_col_x], _[agg_by_col]) 
    #                >> summarize(n=_[sum_col].nunique()) 
    #                >> arrange(-_[groupby_col_x]))

    chart = alt.Chart(df).mark_bar().encode(
            column=(f"{agg_by_col}:N"),
            x=alt.X((f"{groupby_col_x}:O"), title=_dla_utils.labeling(groupby_col_x)),
            y=alt.Y('n:Q', title=(f"Number of Unique {_dla_utils.labeling(sum_col)}")),
            color = alt.Color((f"{agg_by_col}:N"), 
                                  scale=alt.Scale(
                                      range=cp.CALITP_SEQUENTIAL_COLORS),  
                                   legend=alt.Legend(title=(_dla_utils.labeling(sum_col)))
                                  )
                                  )
   # chart.save(f"./chart_outputs/grouped_{agg_by_col}_by_{groupby_col_x}.png")
    
    return chart

In [70]:
dist_years1 = (df >> group_by(_.prepared_y, _.mpo)
               >> summarize(n=_.prefix.nunique()) 
               >> arrange(-_.prepared_y))

In [71]:
dist_years1

,prepared_y,mpo,n
226,"2,023.00",AMBAG,12
227,"2,023.00",BCAG,6
228,"2,023.00",CFCG,3
229,"2,023.00",COFCG,4
230,"2,023.00",FCOG,8
...,...,...,...
257,NaN,SJCG,1
258,NaN,SLAC,1
259,NaN,STPA,1
260,NaN,TCAG,1


In [72]:
df2

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
12,"2,023.00",4,68963358,287372905,513234208,"236,987.48","987,535.76","1,763,691.44",39,80,232,231,201
11,"2,022.00",4,-32264811,342782897,245130910,"-92,982.16","987,846.97","706,429.14",36,96,264,279,252
10,"2,021.00",4,-7581871,151107792,209378112,"-18,674.56","372,186.68","515,709.64",42,101,316,326,288
9,"2,020.00",4,-16015854,219797497,385650237,"-55,418.18","760,544.97","1,334,429.89",34,89,234,243,215
8,"2,019.00",4,-70929401,314851257,319675522,"-177,323.50","787,128.14","799,188.81",41,93,322,333,299
0,"2,018.00",4,-19612826,286154006,374039028,"-43,200.06","630,295.17","823,874.51",51,97,344,369,315
1,"2,017.00",4,-12268569,279804750,332810636,"-33,429.34","762,410.76","906,840.97",43,97,292,319,269
2,"2,016.00",4,59687248,304782100,502320628,"144,172.10","736,188.65","1,213,334.85",50,113,336,361,300
3,"2,015.00",4,68364142,239952075,448655374,"152,258.67","534,414.42","999,232.46",48,109,374,394,300
4,"2,014.00",4,-21667366,178332665,229278381,"-44,491.51","366,186.17","470,797.50",52,108,376,400,295


## Plot Nine

In [73]:
def prefix_all_agencies(df, prefix_unique):
    
    # graphs 
    prefixes = df[df.prefix== prefix_unique]
    
    prefix_count_num = (prefixes >> count(_.primary_agency_name) >> arrange(-_.n)).head(50)
    
    prefix_count = (prefixes >> count(_.primary_agency_name) >> arrange(-_.n)).head(20)
    
    display(Markdown(f"**The number of agencies using {prefix_unique} is {len(prefix_count_num)}**"))
    
    # for the table- using one as some agencies only have one entry
    display(df[(df.prefix == prefix_unique)].sample(1))
    
    chart = (alt.Chart(prefix_count)
             .mark_bar()
             .encode(
                 x=alt.X("primary_agency_name", title="Agency"),
                 y=alt.Y("n", title="Number of Obligations"),
                 #column = "payment:N",
                 color = alt.Color("primary_agency_name", 
                                  scale=alt.Scale(
                                      range=cp.FIVETHIRTYEIGHT_DIVERGING_COLORS
                                  )
                                  )
             )
            .properties( 
                           title="Top Agencies using Prefix")

    )
    
    return chart 
  

In [74]:
prefixes = df[df.prefix== "CML"]
    
prefix_count_num = (prefixes >> count(_.primary_agency_name) >> arrange(-_.n)).head(50)
    
prefix_count = (prefixes >> count(_.primary_agency_name) >> arrange(-_.n)).head(20)

# Misc.

In [77]:
# def totalfunds_chart_(df, aggregate_by=[], col):
#      aggfunc = (
#         geography_utils.aggregate_by_geography(
#             df[df[aggregate_by].notna()], 
#             group_cols = [aggregate_by],
#             sum_cols = [col],
#             nunique_cols = [col]

            
    
#     test= (df
#            >>group_by(_.primary_agency_name)
#            >>summarize(Total_Funds = _[col].sum())
#            >>arrange(-_.Total_Funds))
#     #test.rename(columns={'primary_agency_name': 'Primary Agency Name', 'col': 'Sum Funds'}, inplace=True)
#     test.style.format(precision=2, na_rep='MISSING', thousands=",")
#     chart = (alt.Chart(test.head(20))
#              .mark_bar()
#              .encode(
#                  x=alt.X("primary_agency_name", title="col"),
#                  y=alt.Y("Total Funds", title="Total Funds Obligated"),
#                  #column = "payment:N",
#                  color = alt.Color("primary_agency_name", 
#                                   scale=alt.Scale(
#                                       range=cp.FIVETHIRTYEIGHT_DIVERGING_COLORS
#                                   )
#                                   )
#              )
#             .properties( 
#                            title="District 7 Agencies with the Most Total Funds Obligated")

#     )
#     return chart